In [1]:
# required libraries
# install packages using ----pip install---- command, if required
from datetime import datetime
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm_pandas as tqdmp
from tqdm.notebook import tqdm, tqdm_notebook, trange
from time import sleep
from meteostat import Stations, Daily

In [2]:
# read australia post codes information dataset
australia_long_lat_data = pd.read_csv("Australian_Post_Codes_Lat_Lon.csv")

In [3]:
# filter out suburb if it's empty
australia_long_lat_data = australia_long_lat_data[australia_long_lat_data['suburb'].notna()]

# retrieve Victoria data
victoria_long_lat_data = australia_long_lat_data[australia_long_lat_data.state == 'VIC'].reset_index()

# select first suburb of each postcode
victoria_long_lat_data = victoria_long_lat_data.groupby('postcode').first().reset_index()

# drop unwanted columns
victoria_long_lat_data.drop(['index', 'state', 'dc', 'type'], axis = 1, inplace = True)

In [4]:
# empty dataframe for storing weather information
weather_df = pd.DataFrame(columns=['postcode' ,'suburb', 'lat', 'long', 'tavg','tmin','tmax','prcp','snow','wdir','wspd', 'wpgt', 'pres', 'tsun'])

In [5]:
# looping over each row
for index, row in tqdm_notebook(victoria_long_lat_data.iterrows()):
    
    # getting nearest station information
    stations = Stations(lat = row['lat'], lon = row['lon'])
    station = stations.fetch(1)
    
    # retrieve data for last 6 years
    data = Daily(station, start = datetime(2015, 1, 1), end = datetime(2021, 7, 31))
    data = data.fetch()
    
    data['postcode'] = row['postcode']
    data['suburb'] = row['suburb']
    data['long'] = row['lon']
    data['lat'] = row['lat']
    
    # updating empty dataframe
    weather_df = weather_df.append(data)

In [6]:
# reset index
weather_df.reset_index(level=0, inplace=True)

# rename index to date column
weather_df.rename(columns=({ 'index': 'Date'}), inplace=True)

In [7]:
# split date into year, month and day
weather_df['year'] = pd.DatetimeIndex(weather_df['Date']).year
weather_df['month'] = pd.DatetimeIndex(weather_df['Date']).month
weather_df['day'] = pd.DatetimeIndex(weather_df['Date']).day

In [8]:
# drop unwanted columns
weather_df.drop(['snow', 'wdir', 'wspd', 'wpgt', 'pres', 'tsun', 'prcp'], axis = 1, inplace = True)

# clean up data where long and lat is 0 (if available)
weather_df = weather_df[(weather_df.long != 0) & (weather_df.lat != 0)]

In [9]:
# group data to get average minimum and maximum temp for each month of each suburb
avg_min_temp_df = weather_df.groupby(['postcode', 'year', 'month', 'suburb', 'long', 'lat'])['tmin'].mean().reset_index()
avg_max_temp_df = weather_df.groupby(['postcode', 'year', 'month', 'suburb', 'long', 'lat'])['tmax'].mean().reset_index()

In [10]:
# merging both dataframes
combined_df = pd.concat([avg_min_temp_df, avg_max_temp_df[['tmax']]], axis=1)

In [11]:
# changing order of columns
cols = combined_df.columns.tolist()
cols = cols[-1:] + cols[:-1]
combined_df = combined_df[cols]

In [12]:
def month_name (row):
    if row['month'] == 1 :
        return 'January'
    if row['month'] == 2:
        return 'February'
    if row['month'] == 3 :
        return 'March'
    if row['month'] == 4:
        return 'April'
    if row['month'] == 5:
        return 'May'
    if row['month'] == 6:
        return 'June'
    if row['month'] == 7:
        return 'July'
    if row['month'] == 8:
        return 'August'
    if row['month'] == 9:
        return 'September'
    if row['month'] == 10:
        return 'October'
    if row['month'] == 11:
        return 'November'
    return 'December'

In [13]:
combined_df['month'] = combined_df.apply (lambda row: month_name(row), axis=1)

In [14]:
def season_name(row):
    if (row['month'] == "January") or (row['month'] == "February") or (row['month'] == "December") :
        return 'Summer'
    if (row['month'] == "March") or (row['month'] == "April") or (row['month'] == "May") :
        return 'Autumn'
    if (row['month'] == "June") or (row['month'] == "July") or (row['month'] == "August") :
        return 'Winter'
    if (row['month'] == "September") or (row['month'] == "October") or (row['month'] == "November") :
        return 'Spring'

In [15]:
combined_df['season'] = combined_df.apply (lambda row: season_name(row), axis=1)

In [16]:
# creating primary key column merging postcode, suburb, year and month
combined_df['primaryKey'] = combined_df['postcode'].map(str) + '-' + combined_df['suburb'] + '-' + combined_df['month'] + '-' + combined_df['year'].map(str)

In [17]:
combined_df = combined_df[['primaryKey', 'postcode', 'year', 'month', 'season', 'suburb', 'long', 'lat', 'tmin', 'tmax']]

In [18]:
combined_df['tmin'] = combined_df['tmin'].round(decimals = 2)
combined_df['tmax'] = combined_df['tmax'].round(decimals = 2)

In [19]:
combined_df

,primaryKey,postcode,year,month,season,suburb,long,lat,tmin,tmax
0,3000-MELBOURNE-January-2015,3000,2015,January,Summer,MELBOURNE,144.970267,-37.814563,16.72,30.58
1,3002-EAST MELBOURNE-January-2015,3002,2015,January,Summer,EAST MELBOURNE,144.987811,-37.816640,16.72,30.58
2,3003-WEST MELBOURNE-January-2015,3003,2015,January,Summer,WEST MELBOURNE,144.941123,-37.806255,16.72,30.58
3,3004-MELBOURNE-January-2015,3004,2015,January,Summer,MELBOURNE,144.976335,-37.837324,16.72,30.58
4,3005-WORLD TRADE CENTRE-January-2015,3005,2015,January,Summer,WORLD TRADE CENTRE,144.954856,-37.822262,16.72,30.58
...,...,...,...,...,...,...,...,...,...,...
45801,8865-SOUTH MELBOURNE-April-2021,8865,2021,April,Autumn,SOUTH MELBOURNE,145.033718,-37.932910,11.58,19.99
45802,8865-SOUTH MELBOURNE-May-2021,8865,2021,May,Autumn,SOUTH MELBOURNE,145.033718,-37.932910,8.28,17.44
45803,8865-SOUTH MELBOURNE-June-2021,8865,2021,June,Winter,SOUTH MELBOURNE,145.033718,-37.932910,6.69,15.16
45804,8865-SOUTH MELBOURNE-July-2021,8865,2021,July,Winter,SOUTH MELBOURNE,145.033718,-37.932910,7.12,14.31


In [20]:
# writing data to csv file
combined_df.to_csv("suburb_generated_weather_data.csv", index=False)